## 선거관리위원회 크롤링

In [20]:
import pandas as pd, numpy as np, scipy as sp, seaborn as sns
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import random
import platform
from matplotlib import rc
import glob
from selenium import webdriver
import time

%matplotlib inline

rc('font', family='AppleGothic')

In [29]:
driver = webdriver.Chrome('.../driver/chromedriver')
driver.get('http://info.nec.go.kr/main/showDocument.xhtml?electionId=0020200415&topMenuId=VC&secondMenuId=VCCP08')

In [30]:
xpath = '//*[@id="electionId2"]'
driver.find_element_by_xpath(xpath).click()

In [31]:
def export_data():
    #html parsing
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    #선거구역정보 & 파일이름
    vote_type = soup.find_all('span', id='electionName')[0].get_text().replace('[','').replace(']','')
    region1 = soup.find_all('span', id='cityName')[0].get_text().replace('[','').replace(']','')
    region2_type = soup.find_all('span', id='sggCityName')[0].get_text().replace('[','').replace(']','')
    region2 = soup.find_all('span', id='townNameFromSgg')[0].get_text().replace('[','').replace(']','')
    f_name = vote_type+'_'+region1+'_'+region2_type+'_'+region2


    #컬럼명
    head_list_raw = soup.find_all('thead')[0].get_text().strip().split('\n\n\n')
    head_list_candi = head_list_raw[1].split('\n')

    head_list_raw = head_list_raw[0].split('\n')
    head_list_main = head_list_raw[:4]
    head_list_sub = head_list_raw[-2:]

    head_list = head_list_main + head_list_candi + head_list_sub
    head_list


    #데이터프레임 생성
    df_raw = pd.DataFrame(columns = head_list)


    #df_raw_info
    df_raw['읍면동명'] = [v.get_text() for v in soup.find_all('td', class_='firstTd alignL')]

    tmp_list = []

    for i_soup in soup.find_all('td', class_='alignL'):
        x = str(i_soup)
        if 'firstTd alignL' in x:
            continue
        else:
            tmp_list.append(i_soup.get_text())

    df_raw['투표구명'] = tmp_list
    df_raw_info = df_raw.iloc[:,0:2].copy()


    #df_raw_allvalues
    elect_allvalues = [v.get_text() for v in soup.find_all('td', class_='alignR')]
    elect_allvalues

    df_raw_allvalues = df_raw.iloc[:,2:].copy()
    df_raw_allvalues.shape

    for i in range(0,df_raw_allvalues.shape[0]):
        #print('{}번째'.format(i))
        for j in range(0,df_raw_allvalues.shape[1]):
            num = j+(i*df_raw_allvalues.shape[1])
            df_raw_allvalues.iloc[i,j] = elect_allvalues[num]
            #print(j)


    #df_areainfo
    df_areainfo = pd.DataFrame(columns = ['vote_type','region1','region2_type','region2'])

    #합치기
    df = pd.concat([df_areainfo,df_raw_info,df_raw_allvalues], axis=1)
    df.loc[:,'vote_type'] = vote_type
    df.loc[:,'region1'] = region1
    df.loc[:,'region2_type'] = region2_type
    df.loc[:,'region2'] = region2

    #Export
    export_path = '/Users/hyungkeun/Desktop/21_congress/{}.xlsx'.format(f_name)
    df.to_excel(export_path,index=False) 
    

In [32]:
#시도
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name('option')
sido_names_values = [option.text for option in sido_list]
sido_values = sido_names_values[1:]

for sido in range(1,len(sido_values)+1):
    element = driver.find_element_by_id('cityCode')
    element.send_keys(sido_names_values[sido])
    time.sleep(1)
    
    #선거구
    electarea_list_raw = driver.find_element_by_xpath("""//*[@id="sggCityCode"]""")
    electarea_list = electarea_list_raw.find_elements_by_tag_name('option')
    electarea_values = [option.text for option in electarea_list]
    electarea_values = electarea_values[1:]
    
    for electarea in range(0,len(electarea_values)):
        element = driver.find_element_by_id('sggCityCode')
        element.send_keys(electarea_values[electarea])
        time.sleep(1)
        
        #시군구
        sigungu_list_raw = driver.find_element_by_xpath("""//*[@id="townCodeFromSgg"]""")
        sigungu_list = sigungu_list_raw.find_elements_by_tag_name('option')
        sigungu_values = [option.text for option in sigungu_list]
        sigungu_values = sigungu_values[1]
        sigungu_values

        element = driver.find_element_by_id('townCodeFromSgg')
        element.send_keys(sigungu_values)
        
        #검색
        driver.find_element_by_xpath("""//*[@id="spanSubmit"]/input""").click()
        time.sleep(1)
        
        export_data()

print('Done :', sido, end='/')        

In [148]:
file_list = glob.glob('/Users/hyungkeun/Desktop/21_congress/*.xlsx')

In [155]:
df_prc_list = []
column_names = ['vote_type','region1','region2_type','region2','region3','voteplace','num_electors','num_votes',
 'candi_01','candi_02','candi_03','candi_04','candi_05','candi_06','candi_07','candi_08','candi_09','candi_10','candi_11','candi_12',
 'candi_13','candi_14','candi_15','candi_16','candi_17','candi_18','candi_19','candi_20','candi_21','tot_candi','invalid','abstention']
mynum = 1

for file in file_list:
    #Load
    df_raw = pd.read_excel(file)
    
    #앞부분
    df_raw_head = df_raw.iloc[:,0:8].copy()

    #마지막
    df_raw_tail = df_raw.iloc[:,-3:].copy()

    #후보자
    df_raw_candi = df_raw.iloc[:,8:-3].copy()
    
    #선거구마다 컬럼개수를 맞추기 위해 컬럼 추가
    new_colnames = []
    num = 21 - df_raw_candi.shape[1]

    for i in range(1,num+1):
        name = 'New'+str(i)
        new_colnames.append(name)

    for name in new_colnames:
        df_raw_candi.loc[:,name] = 0
    
    #병합
    df_prc = pd.concat([df_raw_head, df_raw_candi, df_raw_tail], axis=1)
    
    #컬럼명 업데이트
    df_prc.columns = column_names
    
    df_prc_list.append(df_prc)

    #print('Done: ',mynum,end=' /')
    #mynum += 1

Done:  1 /Done:  2 /Done:  3 /Done:  4 /Done:  5 /Done:  6 /Done:  7 /Done:  8 /Done:  9 /Done:  10 /Done:  11 /Done:  12 /Done:  13 /Done:  14 /Done:  15 /Done:  16 /Done:  17 /Done:  18 /Done:  19 /Done:  20 /Done:  21 /Done:  22 /Done:  23 /Done:  24 /Done:  25 /Done:  26 /Done:  27 /Done:  28 /Done:  29 /Done:  30 /Done:  31 /Done:  32 /Done:  33 /Done:  34 /Done:  35 /Done:  36 /Done:  37 /Done:  38 /Done:  39 /Done:  40 /Done:  41 /Done:  42 /Done:  43 /Done:  44 /Done:  45 /Done:  46 /Done:  47 /Done:  48 /Done:  49 /Done:  50 /Done:  51 /Done:  52 /Done:  53 /Done:  54 /Done:  55 /Done:  56 /Done:  57 /Done:  58 /Done:  59 /Done:  60 /Done:  61 /Done:  62 /Done:  63 /Done:  64 /Done:  65 /Done:  66 /Done:  67 /Done:  68 /Done:  69 /Done:  70 /Done:  71 /Done:  72 /Done:  73 /Done:  74 /Done:  75 /Done:  76 /Done:  77 /Done:  78 /Done:  79 /Done:  80 /Done:  81 /Done:  82 /Done:  83 /Done:  84 /Done:  85 /Done:  86 /Done:  87 /Done:  88 /Done:  89 /Done:  90 /Done:  91 /Done:  9

In [156]:
#병합
df = pd.concat(df_prc_list)
df = df.reset_index(drop=True)

In [159]:
# #Export
# df.to_excel('/Users/hyungkeun/Desktop/21대국회의원_득표율.xlsx', index=False)